### Step 1: Import Necessary Libraries

In [1]:
# Step 1: Import Necessary Libraries
from flask import Flask, render_template, request
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer

### Step 2: Initialize the Flask Application

In [2]:
# Step 2: Initialize the Flask Application
app = Flask(__name__)

### Step 3: Load and Prepare the Dataset

In [3]:
# Step 3: Load and Prepare the Dataset
data = pd.read_csv("recipes.csv")

### Step 4: Preprocess Ingredients (TF-IDF Vectorization)

In [4]:
# Step 4: Preprocess Ingredients (TF-IDF Vectorization)
# Convert the list of ingredients into numerical features using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english', max_features=500)
X_ingredients = vectorizer.fit_transform(data['ingredients_list'])

### Step 5: Normalize Numerical Features

In [5]:
# Step 5: Normalize Numerical Features
# Standardize the nutritional data to have zero mean and unit variance
scaler = StandardScaler()
X_numerical = scaler.fit_transform(data[['calories', 'fat', 'carbohydrates', 'protein', 'cholesterol', 'sodium', 'fiber']])

### Step 6: Combine Text and Numerical Features

In [6]:
# Step 6: Combine Text and Numerical Features
# Merge the normalized numerical features and the TF-IDF ingredient features into a single array
X_combined = np.hstack([X_numerical, X_ingredients.toarray()])

### Step 7: Train the KNN Model

In [7]:
# Step 7: Train the KNN Model
# Use the Nearest Neighbors algorithm to find similar recipes
# Set `n_neighbors` to 5 for top 5 recommendations
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X_combined)

NearestNeighbors(metric='euclidean')

### Step 8: Define the Recommendation Function

In [8]:
# Step 8: Define the Recommendation Function
def recommend_recipes(input_features):
    """
    Recommend recipes based on user input features.
    
    Args:
    - input_features (list): List containing nutritional values and ingredient text.
    
    Returns:
    - pd.DataFrame: Top 5 recommended recipes with their details.
    """
    # Step 8.1: Normalize user input nutritional features
    input_features_scaled = scaler.transform([input_features[:7]])
    
    # Step 8.2: Transform user input ingredients into TF-IDF vector
    input_ingredients_transformed = vectorizer.transform([input_features[7]])
    
    # Step 8.3: Combine normalized and TF-IDF features
    input_combined = np.hstack([input_features_scaled, input_ingredients_transformed.toarray()])
    
    # Step 8.4: Get nearest neighbors (top 5 recipes)
    distances, indices = knn.kneighbors(input_combined)
    
    # Step 8.5: Return recommended recipes with relevant details
    recommendations = data.iloc[indices[0]]
    return recommendations[['recipe_name', 'ingredients_list', 'preparation_steps', 'image_url']].head(5)


### Step 9: Utility Function for Truncating Text

In [9]:
# Step 9: Utility Function for Truncating Text
def truncate(text, length=30):
    """
    Truncate text to a specified length with ellipsis.
    
    Args:
    - text (str): The text to be truncated.
    - length (int): Maximum allowed length of the text.
    
    Returns:
    - str: Truncated text with ellipsis if necessary.
    """
    if len(text) > length:
        return text[:length] + "..."
    else:
        return text


### Step 10: Define Flask Routes

In [10]:
# Step 10: Define Flask Routes
@app.route('/', methods=['GET', 'POST'])
def index():
    """
    Handle the root route for displaying and processing recipe recommendations.
    
    Handles:
    - GET requests: Render the initial form with no recommendations.
    - POST requests: Process user input and display recommendations.
    """
    if request.method == 'POST':
        try:
            # Step 10.1: Extract numerical input from the user
            calories = float(request.form['calories'])
            fat = float(request.form['fat'])
            carbohydrates = float(request.form['carbohydrates'])
            protein = float(request.form['protein'])
            cholesterol = float(request.form['cholesterol'])
            sodium = float(request.form['sodium'])
            fiber = float(request.form['fiber'])
            
            # Step 10.2: Extract ingredient input from the user
            ingredients = request.form['ingredients']
        except ValueError:
            # Handle invalid input gracefully
            return render_template('index.html', recommendations=[], error="Please enter valid numerical values.")
        
        # Step 10.3: Combine user input into a single feature list
        input_features = [calories, fat, carbohydrates, protein, cholesterol, sodium, fiber, ingredients]
        
        # Step 10.4: Get recipe recommendations based on user input
        recommendations = recommend_recipes(input_features)
        
        # Step 10.5: Render the template with recommendations
        return render_template(
            'index.html',
            recommendations=recommendations.to_dict(orient='records'),
            truncate=truncate
        )
    
    # Step 10.6: Render the form without recommendations (initial page load)
    return render_template('index.html', recommendations=[])


### Step 11: Run the Application

In [11]:
# Step 11: Run the Application
if __name__ == '__main__':
    # Run the Flask application in debug mode
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
